In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile6 = pd.read_pickle("FFF_Train_Decile6.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile6.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile6, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 6')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 10s 3ms/step - loss: 0.3446 - acc: 0.8717 - val_loss: 0.3057 - val_acc: 0.8901
Epoch 36/75
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3481 - acc: 0.8628 - val_loss: 0.3052 - val_acc: 0.8914
Epoch 37/75
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3391 - acc: 0.8661 - val_loss: 0.3025 - val_acc: 0.8887
Epoch 38/75
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3477 - acc: 0.8678 - val_loss: 0.3027 - val_acc: 0.8927
Epoch 39/75
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3330 - acc: 0.8746 - val_loss: 0.3012 - val_acc: 0.8914
Epoch 40/75
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3385 - acc: 0.8697 - val_loss: 0.2995 - val_acc: 0.8927
Epoch 41/75
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3355 - acc: 0.8694 - val_loss: 0.2992 - val_acc: 0.8914
Epoch 42/75
3055/3055 [==============================] - 10s

3055/3055 [==============================] - 10s 3ms/step - loss: 0.3040 - acc: 0.8769 - val_loss: 0.2835 - val_acc: 0.8901
Epoch 20/75
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3057 - acc: 0.8746 - val_loss: 0.2784 - val_acc: 0.8940
Epoch 21/75
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3102 - acc: 0.8756 - val_loss: 0.2791 - val_acc: 0.8940
Epoch 22/75
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3043 - acc: 0.8782 - val_loss: 0.2777 - val_acc: 0.8940
Epoch 23/75
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3075 - acc: 0.8746 - val_loss: 0.2799 - val_acc: 0.8940
Epoch 24/75
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3006 - acc: 0.8789 - val_loss: 0.2796 - val_acc: 0.8940
Epoch 25/75
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3034 - acc: 0.8730 - val_loss: 0.2804 - val_acc: 0.8927

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.

3055/3055 [==============================] - 10s 3ms/step - loss: 0.4649 - acc: 0.7957 - val_loss: 0.4056 - val_acc: 0.8482
Epoch 3/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.4211 - acc: 0.8426 - val_loss: 0.3813 - val_acc: 0.8508
Epoch 4/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.4017 - acc: 0.8445 - val_loss: 0.3672 - val_acc: 0.8508
Epoch 5/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3821 - acc: 0.8422 - val_loss: 0.3689 - val_acc: 0.8390
Epoch 6/100
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3724 - acc: 0.8606 - val_loss: 0.3329 - val_acc: 0.8691
Epoch 7/100
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3607 - acc: 0.8628 - val_loss: 0.3331 - val_acc: 0.8613
Epoch 8/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3538 - acc: 0.8583 - val_loss: 0.3127 - val_acc: 0.8822
Epoch 9/100
3055/3055 [==============================] - 10s

Epoch 61/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2922 - acc: 0.8802 - val_loss: 0.2752 - val_acc: 0.8940
Epoch 62/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2989 - acc: 0.8773 - val_loss: 0.2755 - val_acc: 0.8953
Epoch 63/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2966 - acc: 0.8759 - val_loss: 0.2753 - val_acc: 0.8940
Epoch 64/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3027 - acc: 0.8776 - val_loss: 0.2751 - val_acc: 0.8940
Epoch 65/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2986 - acc: 0.8815 - val_loss: 0.2752 - val_acc: 0.8940
Epoch 66/100
3055/3055 [==============================] - 11s 3ms/step - loss: 0.2972 - acc: 0.8812 - val_loss: 0.2752 - val_acc: 0.8940
Epoch 67/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2967 - acc: 0.8763 - val_loss: 0.2753 - val_acc: 0.8940
Epoch 68/100
3055/3055 [=================

...

3055/3055 [==============================] - 10s 3ms/step - loss: 0.3689 - acc: 0.8609 - val_loss: 0.3323 - val_acc: 0.8743
Epoch 24/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3627 - acc: 0.8661 - val_loss: 0.3307 - val_acc: 0.8730
Epoch 25/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3592 - acc: 0.8674 - val_loss: 0.3265 - val_acc: 0.8835
Epoch 26/125
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3611 - acc: 0.8645 - val_loss: 0.3243 - val_acc: 0.8822
Epoch 27/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3580 - acc: 0.8678 - val_loss: 0.3218 - val_acc: 0.8861
Epoch 28/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3520 - acc: 0.8661 - val_loss: 0.3198 - val_acc: 0.8835
Epoch 29/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3521 - acc: 0.8642 - val_loss: 0.3163 - val_acc: 0.8874
Epoch 30/125
3055/3055 [==============================

3055/3055 [==============================] - 11s 3ms/step - loss: 0.3140 - acc: 0.8789 - val_loss: 0.2875 - val_acc: 0.8979
Epoch 83/125
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3185 - acc: 0.8759 - val_loss: 0.2876 - val_acc: 0.8966
Epoch 84/125
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3146 - acc: 0.8746 - val_loss: 0.2875 - val_acc: 0.8966
Epoch 85/125
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3135 - acc: 0.8756 - val_loss: 0.2876 - val_acc: 0.8966
Epoch 86/125
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3147 - acc: 0.8750 - val_loss: 0.2870 - val_acc: 0.8979
Epoch 87/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3197 - acc: 0.8714 - val_loss: 0.2871 - val_acc: 0.8979
Epoch 88/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3107 - acc: 0.8805 - val_loss: 0.2870 - val_acc: 0.8966
Epoch 89/125
3055/3055 [==============================

3055/3055 [==============================] - 11s 4ms/step - loss: 0.3155 - acc: 0.8746 - val_loss: 0.2826 - val_acc: 0.8979
Epoch 17/100
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3134 - acc: 0.8743 - val_loss: 0.2849 - val_acc: 0.8979
Epoch 18/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3057 - acc: 0.8753 - val_loss: 0.2778 - val_acc: 0.8966
Epoch 19/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3065 - acc: 0.8753 - val_loss: 0.2838 - val_acc: 0.8979
Epoch 20/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3117 - acc: 0.8700 - val_loss: 0.2851 - val_acc: 0.8966

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 21/100
3055/3055 [==============================] - 11s 3ms/step - loss: 0.3026 - acc: 0.8782 - val_loss: 0.2805 - val_acc: 0.8992
Epoch 22/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3067 - acc: 0.8720 - val_los

3055/3055 [==============================] - 9s 3ms/step - loss: 0.3060 - acc: 0.8727 - val_loss: 0.2766 - val_acc: 0.8979
Epoch 75/100
3055/3055 [==============================] - 8s 2ms/step - loss: 0.3021 - acc: 0.8753 - val_loss: 0.2767 - val_acc: 0.8979
Epoch 76/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3104 - acc: 0.8736 - val_loss: 0.2770 - val_acc: 0.8979
Epoch 77/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3064 - acc: 0.8756 - val_loss: 0.2771 - val_acc: 0.8992
Epoch 78/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3078 - acc: 0.8746 - val_loss: 0.2771 - val_acc: 0.8992
Epoch 79/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3061 - acc: 0.8736 - val_loss: 0.2771 - val_acc: 0.8979
Epoch 80/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.3037 - acc: 0.8710 - val_loss: 0.2771 - val_acc: 0.8979
Epoch 81/100
3055/3055 [==============================] 

Epoch 34/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3080 - acc: 0.8743 - val_loss: 0.2808 - val_acc: 0.8953
Epoch 35/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3076 - acc: 0.8727 - val_loss: 0.2837 - val_acc: 0.8979
Epoch 36/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3060 - acc: 0.8753 - val_loss: 0.2786 - val_acc: 0.8953
Epoch 37/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3005 - acc: 0.8736 - val_loss: 0.2796 - val_acc: 0.8953
Epoch 38/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2968 - acc: 0.8730 - val_loss: 0.2770 - val_acc: 0.8940
Epoch 39/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3008 - acc: 0.8736 - val_loss: 0.2776 - val_acc: 0.8940
Epoch 40/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2999 - acc: 0.8746 - val_loss: 0.2791 - val_acc: 0.8966
Epoch 41/50
3055/3055 [==============================] 

In [3]:
0.93484

0.93484